In [1]:
from pathlib import Path
import sys

home_path = str(Path.cwd().parent)
sys.path.append(home_path)

sys.path.append('/eos/home-i03/m/morwat/.local/lib/python3.9/site-packages/')

In [2]:
from aper_package.interactive_tool import InteractiveTool

In [3]:
tool = InteractiveTool()
tool.show()

Loading line from dict:   0%|          | 0/105823 [00:00<?, ?it/s]

Done loading line from dict.           


Loading line from dict:   0%|          | 0/105797 [00:00<?, ?it/s]

Done loading line from dict.           


In [69]:
import pandas as pd
import numpy as np
import tfs
df_no_nans = tool.aperture_data.aper_b1.dropna()
df_b1 = tfs.read('/eos/project-i02/c/collimation-team/machine_configurations/LHC_run3/2023/MADX/injection/aperture/ap_ir5b1.tfs')
df_b1['NAME']=df_b1['NAME'].str.lower()
df_no_nans['NAME']=df_no_nans['NAME'].str.lower()

/tmp/ipykernel_2487/3405699162.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [70]:
merged = pd.merge(df_no_nans, df_b1, on='NAME')
merged['N1x_custom'] = (merged['APER_1_x']-merged['RTOL']-merged['XTOL']-0.002)/(np.sqrt(1.2*merged['BETX']*tool.aperture_data.epsilon+(merged['DX']**2)*(0.00086**2)))
merged['N1y_custom'] = (merged['APER_2_x']-merged['RTOL']-merged['YTOL']-0.002)/(np.sqrt(1.2*merged['BETY']*tool.aperture_data.epsilon+(merged['DY']**2)*(0.00086**2)))

In [ ]:
merged[['NAME', 'N1', 'N1x_custom', 'N1y_custom']].iloc[10:]